In [1]:
GITHUB_USERNAME = "$GITHUB_USERNAME$"
GITHUB_REF = "$GITHUB_REF$"
NOTEBOOK_TYPE = "$NOTEBOOK_TYPE$"
PYTHON_VERSION = "$PYTHON_VERSION$"
IPYTHON_VERSION = "$IPYTHON_VERSION$"

In [3]:
from pathlib import Path

import requests


if NOTEBOOK_TYPE == 'colab':
    # utils module doesn't exist on colab VM, so get current version from GitHub
    utils_module = Path('utils.py').resolve()
    response = requests.get(f'https://raw.githubusercontent.com/{GITHUB_USERNAME}/davos/{GITHUB_REF}/tests/utils.py')
    utils_module.write_text(response.text)
    # also need to install davos locally
    from utils import install_davos
    install_davos(source='github', ref=GITHUB_REF, fork=GITHUB_USERNAME)

In [6]:
import json
import subprocess
import sys

import davos
import pkg_resources

from utils import mark, run_tests, TestingEnvironmentError

# tests for general testing environment

In [ ]:
def test_expected_python_version():
    installed_version = '.'.join(map(str, sys.version_info[:2]))
    expected_version = PYTHON_VERSION
    if installed_version != expected_version:
        raise TestingEnvironmentError(
            f"Test environment has Python {sys.version.split()[0]}, expected "
            "{PYTHON_VERSION}"
        )

In [ ]:
@mark.jupyter
def test_notebook_using_kernel_python():
    if not sys.executable.endswith('envs/kernel-env/bin/python'):
        raise TestingEnvironmentError(
            "Notebook does not appear to be using the correct python "
            "executable. Expected a path ending in "
            f"'envs/kernel-env/bin/python', found {sys.executable}"
        )

In [ ]:
@mark.skipif(IPYTHON_VERSION == 'latest', reason="runs when IPYTHON_VERSION == 'latest'")
def test_expected_ipython_version():
    try:
        pkg_resources.get_distribution(f"IPython=={IPYTHON_VERSION}")
    except pkg_resources.VersionConflict as e:
        raise TestingEnvironmentError(
            f"Test environment has IPython=={IPython.__version__}, expected "
            f"{IPYTHON_VERSION}") from e

In [ ]:
@mark.skipif(IPYTHON_VERSION != 'latest', reason="runs when IPYTHON_VERSION == 'latest'")
def test_latest_ipython_version():
    pip_exe = davos.config.pip_executable
    outdated_pkgs = subprocess.check_output(
            [pip_exe, 'list', '--outdated', '--format', 'json'], encoding='utf-8'
        )
    outdated_pkgs_json = json.loads(outdated_pkgs)
    for pkg in outdated_pkgs_json:
        if pkg['name'] == 'ipython':
            raise TestingEnvironmentError(
                f"Test environment has IPython=={pkg['version']}, expected "
                f"latest version (IPython=={pkg['latest_version']})"
            )